In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from typing import Optional, List, Any
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import requests


In [6]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
def load_index_and_meta(index_path, meta_path):
    index = faiss.read_index(index_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    return index, meta

def search_faiss(query, index, meta, top_k=2):
    query_vec = embedder.encode([query])
    query_vec = np.array(query_vec).astype("float32")
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        if idx < len(meta):
            results.append(meta[idx])
    return results
class ollama(LLM):
    model_name: str = "mistral"

    @property
    def _llm_type(self) -> str:
        return "ollama"
    
    def _call(self,prompt: str, stop: Optional[List[str]] = None, **kwargs: Any):
        response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={"model": self.model_name, "prompt": prompt, "stream": False}
    )
        return response.json()["response"]

In [10]:
ipc_index, ipc_meta = load_index_and_meta("ipc.index", "ipc.json")
judg_index, judg_meta = load_index_and_meta("judgments.index", "judgments.json")

In [11]:
custom=PromptTemplate(input_variables=["history","input"],template=""" You are an legal Chatbot assistant specializing in India Penal Code.
                      
                      Previous conversations:{history}
                      
                      Current question and also judgements :{input}
                      
                    provide an suitable reply for the current question by utilizing previous conversations, keep it around 20 to 30 words .
                      Utilize similar case judgement and compare it with the user case summary and reply but do not repeat about judgement in every replies.
                      Only use these given text as knowledge and reply with that knowledge DONT Retrieve your own knowledge
                     Only talk about legal replies DO NOT REPLY ANY QUESTIONS WHICH ARE OUT OF INDIAN PENAL CODE .
                      Don't reply to refer for an legal advisor""")

In [12]:
llm=ollama()
memory=ConversationBufferMemory()
conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)

C:\Users\nihca\AppData\Local\Temp\ipykernel_14452\3123097401.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
C:\Users\nihca\AppData\Local\Temp\ipykernel_14452\3123097401.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversion = ConversationChain(llm=llm,memory=memory,prompt=custom,verbose=False)


In [16]:
print("⚖️ IPC + Judgement Chatbot (type 'exit' to quit)\n")

user_input = input("You: ").strip()


ipc_results = search_faiss(user_input, ipc_index, ipc_meta)
judg_results = search_faiss(user_input, judg_index, judg_meta)

        # Step 2: Build context
context = "Relevant IPC Sections:\n"
for sec in ipc_results:
    context += f"- Section {sec['section_no']}: {sec['Description']} (Punishment: {sec['punishment_raw']})\n"

prompt = f"User Query: {user_input}\n\n{context}\n\nBased on the above IPC sections, Name the sections relating to the user query and also list possible punishments."
Reply_1=llm._call(prompt)

context += "\nRelevant Judgements:\n"
for case in judg_results:
    context += f"""- Case facts: {case['case_facts']}
Issues: {case['issues']}
Arguments: {case['arguments']}
Decision: {case['decision']}
"""



print(f"\nBot: {Reply_1}\n")

    


⚖️ IPC + Judgement Chatbot (type 'exit' to quit)


Bot:  Based on the user's query, it seems like they might have committed an act that could fall under Section 302 of the Indian Penal Code (IPC), which pertains to murder.

Section 302 IPC defines murder as the intentional causing of death or such bodily injury as is likely to cause death, with the intention of causing death, or with the knowledge that it is likely to cause death. The punishment for murder under Section 302 of the IPC is imprisonment for life, which means imprisonment until the convict's death.

However, since the user's query suggests a history of daily torture and ragging, there could potentially be additional charges filed based on other sections as well. The acts of ragging or torturing others are criminal offenses under Section 326 (voluntarily causing grievous hurt) or even Section 307 (attempt to murder), depending on the severity and frequency of the actions mentioned.

It is essential to stress that this respo

In [17]:
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    user_input+=f"\n\n similar case judgements : {context}"

    answer = conversion.predict(input=user_input)

    print(f"\nBot: {answer}\n")


Bot:  In the Indian Penal Code, intention to inflict a bodily injury sufficient to cause death, even if not intended, can lead to murder charges. Multiple injuries causing death due to public enmity have resulted in conviction (e.g., Rompicherla case). However, specific circumstances vary for each case. Consult a legal advisor for advice on your situation.

Bot: Goodbye!
